In [1]:
from langchain_core.embeddings import Embeddings
from typing import Any, Dict, List, Optional
# from sentence_transformers import SentenceTransformer
import warnings
import logging
logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)
warnings.filterwarnings("ignore")

from IPython.display import clear_output
clear_output()


In [3]:
from models.UsersModel import UsersModel
from database import get_db
from services.llm_chains import get_related_products
from services.find_similar import find_similar_product
from langchain.vectorstores import FAISS


In [35]:
class FineTunedHuggingFaceModel(Embeddings):
    def __init__(self, model_name: str):
        """Initialize with the model name or path."""
        self.model_name = model_name
        self.model = SentenceTransformer(model_name, device="cpu")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # Get the embeddings for the documents
        embeddings = self.model.encode(texts, convert_to_tensor=True)

        # Convert the tensor embeddings into a list of lists (each inner list represents an embedding)
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]
    

model_dir=r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\bge_models_weights"
fine_tuned_model = SentenceTransformer(
    model_dir, device="cpu")
embeddings=FineTunedHuggingFaceModel(model_name=model_dir)

In [4]:
def get_similar_product_from_query(query):
    result=get_related_products(query)
    print(result)
    result={"model":result.get("model",{}).get("value",[])}
    new_res=find_similar_product(result)
    return new_res

query="Give me best mobile phone with minimum 4GB ram and 4200mAh batter."
result=get_similar_product_from_query(query)
clear_output()
print("User Query: ",query)
print("Result Obtained: \n",result)

User Query:  Give me best mobile phone with minimum 4GB ram and 4200mAh batter.
Result Obtained: 
 Empty DataFrame
Columns: [model, brand_name, price, avg_rating]
Index: []


In [ ]:

output_vector_loc=r"C:\Users\prabigya\Desktop\work_here\Recommendo-Personalized-Shopping-Powered-by-LLMs\recommendation_system_ecommerce\dataset\vector_store"
loaded_vector_store = FAISS.load_local(output_vector_loc,embeddings,allow_dangerous_deserialization=True)

In [65]:
def get_recommendations(data):
    user_id=data["user_id"]
    db = next(get_db())
    user = db.query(UsersModel).filter(UsersModel.id == user_id).first()
    # user = db.query(UsersModel).all()

    query = f"User preference: {user.likes}"
    results = loaded_vector_store.similarity_search_with_score(query,k=4)
    # Check similarity scores and proceed if greater than threshold
    recommeded_product=[]
    for result, score in results:
        recommeded_product.append(result.metadata.get("product_name",""))
    return user_id,user.likes,recommeded_product

data = {"user_id": 190}
user_id,user_likes,recommeded_products=get_recommendations(data)
clear_output()

print("User Id: ",user_id)
print("User Preference and Likes: ",user_likes)
print("Recommeded Product: ",recommeded_products)

User Id:  190

User Preference and Likes:  school supplies, cameras, mobiles, Appliances

Recommeded Product: 
[
    'Slb 020Rp 10 L Backpack',
    'United Bags Cross Chain 35 L Medium Laptop Backpack',
    'Mobiware Mw4P Mobiware 4000 Mah Pocket Powerbank 4000 Mah',
    'Hp 15-Ac116Tx (Notebook) (Core I3 (5Th Gen)/ 4Gb/ 1Tb/ Win10/ 2Gb Graph) (N8M19Pa)'
]